# Measuring RRDP bandwidth

This analyses the bandwidth measurements from Appendix A. The input files are gathered as follows:

```bash

cargo install \
    --git https://github.com/SIDN/pqc-routinator \
    --branch measuring-rrdp-bandwidth routinator --locked

for i in {1..10} ; do 
    /usr/bin/time -f "%e, %U, %S" -a -o update_time.csv \
        routinator \
            -v --logfile "timing_${i}.log" \
            --fresh --disable-rsync --rrdp-connect-timeout 30 update;
    # Extract content sizes of RPKI objects in snapshots.
    cat timing_${i}.log \
        | grep "size: " \
        | jq -Rs '
            split("\n[") | [
                .[] | split(", ") | {
                    size: (.[0] | split("size: ") | .[1] | tonumber), 
                    session: .[2], 
                    serial: (.[3] | tonumber)
                } 
            ]
        ' > snapshots_${i}.json; 
    # Extract timing of HTTP requests.
    cat timing_${i}.log \
        | grep "timing: " \
        | jq -Rs '
            split("\n[") | [
                .[] | split(", ") | {
                    url: (.[0] | split("timing: ") | .[1]), 
                    time: (.[1] | tonumber)
                } 
            ]
        ' > timings_${i}.json; 
    echo "Done with run $i."; 
done
```

In [1]:
import json

from statistics import mean, stdev, median

data = {"runs": []}
for i in range(1, 11):
    with open(f"snapshots_{i}.json") as f:
        snapshots = json.load(f)
    with open(f"timings_{i}.json") as f:
        timings = json.load(f)

    snap_times = [t for t in timings if "snapshot" in t["url"]]
    for snap in snapshots:
        matching = [t for t in snap_times if snap["session"] in t["url"]]
        if len(matching) == 1:
            snap["url"] = matching[0]["url"]
            snap["time"] = matching[0]["time"]

    data["runs"].append({
        "snapshots": [{"url": snap["url"], "size": snap["size"], "time": snap["time"]} for snap in snapshots],
        "total_time_snapshots": sum(snap["time"] for snap in snapshots),
        "total_time_all": sum(t["time"] for t in timings),
        "total_size_snapshots": sum(snap["size"] for snap in snapshots),
	"max_snapshot_size": max(snap["size"] for snap in snapshots),
	"max_snapshot_time": max(snap["time"] for snap in snapshots),
	"med_snapshot_size": median(snap["size"] for snap in snapshots),
	"med_snapshot_time": median(snap["time"] for snap in snapshots),
    })

data.update({
    "avg_time_snapshots": mean([run["total_time_snapshots"] for run in data["runs"]]),
    "std_time_snapshots": stdev([run["total_time_snapshots"] for run in data["runs"]]),
    "avg_time_notifications": mean([run["total_time_all"] - run["total_time_snapshots"] for run in data["runs"]]),
    "std_time_notifications": stdev([run["total_time_all"] - run["total_time_snapshots"] for run in data["runs"]]),
    "avg_time_all": mean([run["total_time_all"] for run in data["runs"]]),
    "std_time_all": stdev([run["total_time_all"] for run in data["runs"]]),
    "avg_size_snapshots": mean([run["total_size_snapshots"] for run in data["runs"]]),
    "std_size_snapshots": stdev([run["total_size_snapshots"] for run in data["runs"]]),
})

In [2]:
import pandas as pd

df = pd.read_csv("update_time.csv", header=None, names=["wall (s)", "user (s)", "system (s)"])
print(df)

   wall (s)  user (s)  system (s)
0    341.30    103.36      194.66
1    326.51     95.43      180.51
2    298.97     94.46      177.63
3    308.17     95.16      179.45
4    341.22     95.06      179.77
5    320.47     94.69      179.42
6    325.94     95.42      180.17
7    317.30     94.94      176.78
8    336.33     95.83      180.29
9    323.14     95.28      179.43


In [3]:
print("Measurement\tMean time (s)\tStd. dev. (s)")
print("Snapshots\t{:.1f}\t\t{:.1f}".format(data["avg_time_snapshots"]/1_000_000, data["std_time_snapshots"]/1_000_000))
print("Notifications\t{:.1f}\t\t{:.1f}".format(data["avg_time_notifications"]/1_000_000, data["std_time_notifications"]/1_000_000))
print("Total wall time\t{:.1f}\t\t{:.1f}".format(df["wall (s)"].mean(), df["wall (s)"].std()))

Measurement	Mean time (s)	Std. dev. (s)
Snapshots	14.5		1.5
Notifications	29.5		5.6
Total wall time	323.9		13.7
